# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SVMSMOTE, BorderlineSMOTE
from imblearn.over_sampling import RandomOverSampler
import imblearn

[nltk_data] Downloading package punkt to /home/bmartins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bmartins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bmartins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [3]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)

In [4]:
df.head()

,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,related,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
# preliminary cleaning
df.iloc[:,4:] = df.iloc[:,4:].astype('int64')
df.drop(['related','child_alone'], axis=1, inplace=True)

### 2. Write a tokenization function to process your text data

In [6]:
stop_words = set(stopwords.words('english'))
def tokenize(sentence):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

message = df.message.apply(lambda x: " ".join(tokenize(x)))

In [7]:
message[:10][0]

'weather update cold front cuba could pas haiti'

#### Classes are very imbalanced

In [8]:
print(df.shape)
df.iloc[:,4:].apply(lambda x: x.value_counts())

(26216, 38)


,aid_centers,aid_related,buildings,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,25907,15356,24883,25811,25686,25022,21141,23761,25684,25934,24061,23293,25933,24511,24132,24903,25356,25918,25612,26098,22770,25065,24840,25341,21742,25492,25745,23902,26096,23773,26057,25015,24544,18919
1,309,10860,1333,405,530,1194,5075,2455,532,282,2155,2923,283,1705,2084,1313,860,298,604,118,3446,1151,1376,875,4474,724,471,2314,120,2443,159,1201,1672,7297


In [9]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
request                       0
search_and_rescue             0
security                      0
shelter 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
class Clean_text(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
        #return X, y
    
    def tokenize(self, sentence):
        #stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [11]:
clean_text = Clean_text()

clean_text.transform(df.message)[:5]

array(['weather update cold front cuba could pas haiti', 'is hurricane',
       'looking someone name',
       'un report leogane destroyed only hospital st croix functioning needs supply desperately',
       'say west side haiti rest country today tonight'], dtype=object)

In [12]:
ml_pipeline = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('abclf', MultiOutputClassifier(AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X = df.message.copy()
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) # training
X_train.shape, y_train.shape
ml_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Because classes are imbalanced, accuracy is not a good metric for this problem.

In [14]:
X_train.head()

23313    Forty-eight teams of medical and paramedical s...
7683                   NOTES: this message is not complete
23462    All graphs provide evidence that, considering ...
22334    construction material: bricks - 500 t; cement ...
6522     I can arrange shelter shelter, i live out in t...
Name: message, dtype: object

In [15]:
def cross_val(X_train, y_train, pipeline):
    labels = []
    preds = []
    kf = KFold(n_splits=5, shuffle=True)
    for train_indices, test_indices in kf.split(X_train, y_train):
   
        X_train_kf = X_train.iloc[train_indices]
        y_train_kf = y_train[train_indices]
    
        X_test_kf = X_train.iloc[test_indices]
        y_test_kf = y_train[test_indices]
  
        pipeline.fit(X_train_kf, y_train_kf)
    
                
        labels.extend(y_test_kf)
        preds.extend(pipeline.predict(X_test_kf))

    return pd.DataFrame(labels), pd.DataFrame(preds)
    

In [16]:
labels_df, preds_df = cross_val(X_train, y_train, ml_pipeline)

KeyboardInterrupt: 

In [ ]:
def scoring(scoring_metric, labels_df, preds_df):
    return np.mean(np.array([scoring_metric(labels_df.iloc[:,column], preds_df.iloc[:,column]) 
            for column in range(0,len(df.iloc[:,4:].columns))]))

list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

In [ ]:
y_preds = ml_pipeline.predict(X_test)

labels_df = pd.DataFrame(y_test)
preds_df = pd.DataFrame(y_preds)
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

In [ ]:
scoring(f1_score, labels_df, preds_df)

In [ ]:
# sanity check
X_mod = df.message.copy()
y_mod = df.iloc[:,4].values 

ml_pipeline_single = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('abclf', AdaBoostClassifier())])

X_train_mod, X_test_mod, y_train_mod, y_test_mod = train_test_split(X_mod, y_mod, test_size=0.2, random_state=32)
ml_pipeline_single.fit(X_train_mod, y_train_mod)
labels_df_mod, preds_df_mod = cross_val(X_train_mod, y_train_mod, ml_pipeline_single)

In [ ]:
f1_score(labels_df_mod, preds_df_mod)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
ml_pipeline.get_params()

In [27]:
#parameters = {
#    'vectorizer__max_df': [1.0,2.0],
#    'vectorizer__max_features': [None,5,10],
#    'vectorizer__ngram_range': [(1, 1),(1,2)],
#    'tfidf__norm': ['l2','l1'],
#    'tfidf__smooth_idf': [True, False],
#    'tfidf__sublinear_tf': [True, False],
#    'abclf__estimator__learning_rate': [0.1,0.2,0.5],
#    'abclf__estimator__n_estimators': [50,100]
#}

X = df.message.copy()
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) 

parameters = {
    #'vectorizer__min_df': [1],
    #'vectorizer__max_df': [1.0],
    #'vectorizer__max_features': [None],
    #'vectorizer__ngram_range': [(1,1)],
    #'tfidf__norm': ['l2'],
    #'tfidf__smooth_idf': [True],
    #'tfidf__sublinear_tf': [False],
    'abclf__estimator__learning_rate': [0.1,0.2],
    'abclf__estimator__n_estimators': [100]
}


grid_clf = GridSearchCV(ml_pipeline, param_grid=parameters, verbose=10)

grid_clf.fit(X_train, y_train)

/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100 
[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100, score=0.43598912887998853, total= 2.2min
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100, score=0.43827778572450293, total= 2.2min
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.0min remaining:    0.0s


...

[CV]  abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100, score=0.43598912887998853, total= 2.2min
[CV] abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 12.4min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100, score=0.4373390557939914, total= 2.1min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 14.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 14.8min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'abclf__estimator__learning_rate': [0.1, 0.2], 'abclf__estimator__n_estimators': [100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [30]:
ml_pipeline_best = grid_clf.best_estimator_
print(ml_pipeline_best)

Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...r=None,
          learning_rate=0.1, n_estimators=100, random_state=None),
           n_jobs=None))])


In [ ]:
### Oversampling with SMOTE

In [ ]:
class applySMOTE(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        sm = SMOTE(random_state=42, k_neighbors=3, m_neighbors=3)
        return sm.fit_resample(X, y)
    
    def transform(self, X, y=None):
        return sm.fit_resample(X, y)
        

In [ ]:
class applyROS(BaseEstimator, TransformerMixin):
    def __init__(self):
        sm = RandomOverSampler(random_state=42, sampling_strategy='minority')
        
    def fit(self, X=None, y=None):
        return sm.fit_resample(X, y)
    
    def transform(self, X, y=None):
        return sm.fit_resample(X, y)
        

In [ ]:
ml_pipeline_smote = imblearn.pipeline.Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sampling', SMOTE()),
    #('applyROS', applyROS()),
    ('abclf', AdaBoostClassifier())])

In [ ]:
ml_pipeline_smote = imblearn.pipeline.Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sampling', SMOTE())])

In [ ]:
def pipe_smote(X, y):
    clean_text = Clean_text() 
    X = clean_text.transform(X, y)
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(X)
    sampling = SMOTE(random_state=42)
    X_res, y_res = sampling.fit_resample(X, y)
    
    #return X
    return X_res, y_res

In [ ]:
X_res, y_res = pipe_smote(X_train_mod, y_train_mod)

In [ ]:
X_res.shape

In [ ]:
X_res.iloc[1,:]

In [ ]:
#X_res_df = pd.SparseDataFrame(X_res).to_dense()

In [146]:
clf = AdaBoostClassifier()

In [148]:
clf.fit(X_res, y_res)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [149]:
labels = []
preds = []

kf = KFold(n_splits=5, shuffle=True)
for train_indices, test_indices in kf.split(X_res_df, y_res):
   
    X_train_kf = X_train.iloc[train_indices, :]
    y_train_kf = y_train[train_indices]
    
    X_test_kf = X_train.iloc[test_indices, :]
    y_test_kf = y_train[test_indices]
  
    pipeline.fit(X_train_kf, y_train_kf)
    
                
    labels.extend(y_test_kf)
    preds.extend(pipeline.predict(X_test_kf))


IndexError: positional indexers are out-of-bounds

In [117]:
labels_df, preds_df = cross_val(X_res, y_res, clf)

AttributeError: iloc not found

In [65]:
df.iloc[:,4:].apply(lambda x: x.value_counts())

,aid_centers,aid_related,buildings,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,25907,15356,24883,25811,25686,25022,21141,23761,25684,25934,24061,23293,25933,24511,24132,24903,25356,25918,25612,26098,22770,25065,24840,25341,21742,25492,25745,23902,26096,23773,26057,25015,24544,18919
1,309,10860,1333,405,530,1194,5075,2455,532,282,2155,2923,283,1705,2084,1313,860,298,604,118,3446,1151,1376,875,4474,724,471,2314,120,2443,159,1201,1672,7297


In [68]:
labels_df, preds_df = ml_pipeline_smote.fit(X_train_mod, y_train_mod)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 6

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model. Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
ml_pipeline_best = ml_pipeline
ml_pipeline_best.fit(X_train, y_train)
y_preds = ml_pipeline_best.predict(X_test)

labels_df = pd.DataFrame(y_test)
preds_df = pd.DataFrame(y_preds)
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

In [ ]:
preds_df.apply(lambda x: x.value_counts())

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [56]:
class Clean_text_with_stemmer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
    
    def tokenize(self, sentence):
        stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [stemmer.stem(lemmatizer.lemmatize(word).lower().strip()) for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [57]:
ml_pipeline_logistic = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('mbclf', MultiOutputClassifier(LogisticRegression(random_state=34)))])

In [58]:
labels_df, preds_df = cross_val(X_train, y_train, ml_pipeline_logistic)

/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to '

In [69]:
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'f1_score': 0.2821614837472369,
 'accuracy_score': 0.9515839045898734,
 'precision_score': 0.588623845419117,
 'recall_score': 0.21344285802096127}

In [70]:
ml_pipeline_svc = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('mbclf', MultiOutputClassifier(SVC(C=1.0, gamma=0.1)))])

labels_df, preds_df = cross_val(X_train, y_train, ml_pipeline_svc)
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bmartins/.local/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'f1_score': 0.21492064065600983,
 'accuracy_score': 0.9502039133410373,
 'precision_score': 0.4327221417902778,
 'recall_score': 0.17145228507173793}

### 9. Export your model as a pickle file

In [ ]:
filename='data/ml_pipeline_improved.pkl'
pickle.dump(ml_pipeline_improved, open(filename,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Code Bank

In [ ]:
# beautiful solution
#df.message.apply(lambda x: " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).split() 
#          if word not in stop_words]) )

In [ ]:
class Select_features(BaseEstimator, TransformerMixin):
    # constructor
    def __init__(self, column_names):
        self.column_names = column_names 
    
    # pipeline must implement fit
    def fit(self, X=None, y=None):
        return self
    
    # return new dataframe with selected features
    def transform(self, X=None, y=None):
        return X[self.column_names] 

In [ ]:
X = vector_features
#y = MultiLabelBinarizer().fit_transform(labels)
#print(len(labels))
#print(y.shape)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)
#
#vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
#clf = OneVsRestClassifier(BernoulliNB())
#message_pipeline = Pipeline([('vectorizer', vectorizer),('clf',clf)])
#message_pipeline.fit()
#message_pipeline.named_steps['vectorizer'].transform([df_clean_message])